# DataNode Type Counts

In [1]:
from pip._internal import main as pip
try:
    import csv
except ImportError:
    pip(['install', 'csv'])
    import csv
try:
    from SPARQLWrapper import SPARQLWrapper, JSON
except  ImportError:
    pip(['install', 'sparqlwrapper'])
    from SPARQLWrapper import SPARQLWrapper, JSON
    
import pandas
    
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
DataNodeCount = pandas.DataFrame(columns=['DataNode Type', 'DataNode Count'])

pathwayQuery = '''
      SELECT Distinct (substr(str(?o),41) as ?DNType) (COUNT(?o) as ?DNCount)
      WHERE {
        ?entity a ?o . 
        ?entity a wp:DataNode . 
        ?entity dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
         
      } GROUP BY ?o
        ORDER BY DESC (?DNCount)
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	DataNodeCount = DataNodeCount.append({
		'DataNode Type': result["DNType"]["value"],
        'DataNode Count': result["DNCount"]["value"],
	 }, ignore_index=True)
    
DNCountInteger = DataNodeCount.loc[:, "DataNode Count"].apply(pandas.to_numeric, errors = 'ignore').sum() 

In [2]:
DataNodeCount

,DataNode Type,DataNode Count
0,DataNode,27189
1,GeneProduct,21132
2,Protein,8130
3,Metabolite,3970
4,Rna,1201
5,Complex,40


# GPMLRDF datanodes WITHOUT a WPRDF datanode equivalent 

In [3]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
GpmlWoWP = pandas.DataFrame(columns=['DataNode Type', 'DataNode Count'])

pathwayQuery = '''
      SELECT (STR(?typeIRI) AS ?type) (COUNT(?datanode) AS ?count) WHERE {
          ?datanode a gpml:DataNode ; 
            gpml:type ?typeIRI .
          ?datanode dcterms:isPartOf ?pathway .
          ?wpPathway    a wp:Pathway ;
            wp:isAbout ?pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
          MINUS { ?wpDatanode wp:isAbout ?datanode } .
       } GROUP BY ?typeIRI
         ORDER BY DESC(?count)

    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	GpmlWoWP = GpmlWoWP.append({
		'DataNode Type': result["type"]["value"],
        'DataNode Count': result["count"]["value"],
	 }, ignore_index=True)
    
GpmlWoWPInteger = GpmlWoWP.loc[:, "DataNode Count"].apply(pandas.to_numeric, errors = 'ignore').sum() 

In [4]:
GpmlWoWP

,DataNode Type,DataNode Count
0,GeneProduct,1069
1,Pathway,249
2,Metabolite,221
3,Unknown,218
4,Protein,144
5,Rna,66
6,Complex,16


# % GPML RDF datanodes WITHOUT a WP RDF datanode equivalent

In [5]:
percentGPWoWP = ( GpmlWoWPInteger / (GpmlWoWPInteger + DNCountInteger) ) * 100
formPercentGPWoWP = str(round(percentGPWoWP, 3)) + '% of GPML datanodes WITHOUT a WP datanode equivalent'

In [6]:
formPercentGPWoWP

'3.116% of GPML datanodes WITHOUT a WP datanode equivalent'

#  The Complex GPMLRDF datanodes without WPRDF equivalents

In [7]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
CompGpmlWoWp = pandas.DataFrame(columns=['Complex DataNodes'])

pathwayQuery = '''
      SELECT  (substr(str(?datanode),37 ) as ?datanodes ) ?label
      WHERE {
          ?datanode a gpml:DataNode ; gpml:type "Complex"^^xsd:string .
          ?datanode dcterms:isPartOf ?pathway .
          ?wpPathway    a wp:Pathway ;
            wp:isAbout ?pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
          MINUS { ?wpDatanode wp:isAbout ?datanode }
          OPTIONAL {?datanode gpml:textlabel ?label . }
          
        }

    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	CompGpmlWoWp = CompGpmlWoWp.append({
		'Complex DataNodes': result["datanodes"]["value"],
        'Complex Labels': result["label"]["value"],
	 }, ignore_index=True)

In [8]:
CompGpmlWoWp

,Complex DataNodes,Complex Labels
0,WP2586_r91687/DataNode/b328d,DRE region
1,WP2586_r91687/DataNode/c3d69,TATA
2,WP391_r71373/DataNode/fc7a3,PKA
3,WP2806_r103008/DataNode/a680a,C1q
4,WP2806_r103008/DataNode/b45a6,C3bB3bP
5,WP2806_r103008/DataNode/e4901,C3bB3b
6,WP2806_r103008/DataNode/f51e9,C4b2b
7,WP2806_r103008/DataNode/f7d7c,C3bBbP
8,WP2806_r103008/DataNode/fd0cb,C3bBb
9,WP2806_r103008/DataNode/fd947,C4b2b3b
